<a href="https://colab.research.google.com/github/sharlynmuturi/data-engineering-and-analytics-projects/blob/main/customer-shopping-behavior-analysis/shopping_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd

df = pd.read_csv('/content/customer_shopping_behavior.csv')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3900 entries, 0 to 3899
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Customer ID             3900 non-null   int64  
 1   Age                     3900 non-null   int64  
 2   Gender                  3900 non-null   object 
 3   Item Purchased          3900 non-null   object 
 4   Category                3900 non-null   object 
 5   Purchase Amount (USD)   3900 non-null   int64  
 6   Location                3900 non-null   object 
 7   Size                    3900 non-null   object 
 8   Color                   3900 non-null   object 
 9   Season                  3900 non-null   object 
 10  Review Rating           3863 non-null   float64
 11  Subscription Status     3900 non-null   object 
 12  Shipping Type           3900 non-null   object 
 13  Discount Applied        3900 non-null   object 
 14  Promo Code Used         3900 non-null   

In [7]:
df.isnull().sum()

,0
Customer ID,0
Age,0
Gender,0
Item Purchased,0
Category,0
Purchase Amount (USD),0
Location,0
Size,0
Color,0
Season,0


In [8]:
df['Review Rating'] = df.groupby('Category')['Review Rating'].transform(lambda x: x.fillna(x.median()))

In [9]:
df.isnull().sum()

,0
Customer ID,0
Age,0
Gender,0
Item Purchased,0
Category,0
Purchase Amount (USD),0
Location,0
Size,0
Color,0
Season,0


In [10]:
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(' ', '_')

In [11]:
df.columns

Index(['customer_id', 'age', 'gender', 'item_purchased', 'category',
       'purchase_amount_(usd)', 'location', 'size', 'color', 'season',
       'review_rating', 'subscription_status', 'shipping_type',
       'discount_applied', 'promo_code_used', 'previous_purchases',
       'payment_method', 'frequency_of_purchases'],
      dtype='object')

In [13]:
df = df.rename(columns={'purchase_amount_(usd)':'purchase_amount'})
df.columns

Index(['customer_id', 'age', 'gender', 'item_purchased', 'category',
       'purchase_amount', 'location', 'size', 'color', 'season',
       'review_rating', 'subscription_status', 'shipping_type',
       'discount_applied', 'promo_code_used', 'previous_purchases',
       'payment_method', 'frequency_of_purchases'],
      dtype='object')

In [14]:
# Creating age group column
labels = ['Young Adult', 'Adult', 'Middle-Aged', 'Senior']
df['age_group'] = pd.qcut(df['age'], q=4, labels=labels)

df[['age', 'age_group']].head(10)

,age,age_group
0,55,Middle-Aged
1,19,Young Adult
2,50,Middle-Aged
3,21,Young Adult
4,45,Middle-Aged
5,46,Middle-Aged
6,63,Senior
7,27,Young Adult
8,26,Young Adult
9,57,Middle-Aged


In [15]:
# Creating Purchase frequency in days ie no. of days btwn purchases
df['frequency_of_purchases'].unique()

array(['Fortnightly', 'Weekly', 'Annually', 'Quarterly', 'Bi-Weekly',
       'Monthly', 'Every 3 Months'], dtype=object)

In [16]:
frequency_mapping = {
    'Weekly': 7,
    'Fortnightly': 14,
    'Bi-Weekly': 14,
    'Monthly': 30,
    'Quaterly:': 90,
    'Every 3 Months':90,
    'Annualy': 365
  }

df['purchase_frequency_days'] = df['frequency_of_purchases'].map(frequency_mapping)

df[['purchase_frequency_days', 'frequency_of_purchases']].head(10)

,purchase_frequency_days,frequency_of_purchases
0,14.0,Fortnightly
1,14.0,Fortnightly
2,7.0,Weekly
3,7.0,Weekly
4,NaN,Annually
5,7.0,Weekly
6,NaN,Quarterly
7,7.0,Weekly
8,NaN,Annually
9,NaN,Quarterly


In [18]:
(df['discount_applied'] == df['promo_code_used']).all()

np.True_

In [19]:
df = df.drop('promo_code_used', axis = 1)

In [30]:
import sqlite3
import pandas as pd

engine = sqlite3.connect("shopping.db")

table_name = 'shopping'
df.to_sql(table_name, engine, if_exists="replace", index=False)

print(f"Data successfully loaded into table {table_name} in database {database}.")


Data successfully loaded into table shopping in database ShoppingBehavior.


In [31]:
pd.read_sql_query("SELECT * FROM shopping", engine).head()

,customer_id,age,gender,item_purchased,category,purchase_amount,location,size,color,season,review_rating,subscription_status,shipping_type,discount_applied,previous_purchases,payment_method,frequency_of_purchases,age_group,purchase_frequency_days
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Express,Yes,14,Venmo,Fortnightly,Middle-Aged,14.0
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Express,Yes,2,Cash,Fortnightly,Young Adult,14.0
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Free Shipping,Yes,23,Credit Card,Weekly,Middle-Aged,7.0
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,Next Day Air,Yes,49,PayPal,Weekly,Young Adult,7.0
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Free Shipping,Yes,31,PayPal,Annually,Middle-Aged,NaN


In [32]:
# Total revenue generated by male vs female

pd.read_sql_query("""
SELECT
  gender,
  SUM(purchase_amount) AS revenue
  FROM shopping
  GROUP BY gender;
  """, engine)

,gender,revenue
0,Female,75191
1,Male,157890


In [36]:
# Customers that used a discount but spent more than avg purchase amt

pd.read_sql_query("""
SELECT
  customer_id,
  purchase_amount
  FROM shopping
  WHERE discount_applied = 'Yes' AND purchase_amount > (SELECT AVG(purchase_amount) FROM shopping)
  """, engine).head()

,customer_id,purchase_amount
0,2,64
1,3,73
2,4,90
3,7,85
4,9,97


In [38]:
# Top 5 products in ratings
pd.read_sql_query("""
SELECT
  item_purchased,
  AVG(review_rating) AS "Average Product Rating"
  FROM shopping
  GROUP BY item_purchased
  ORDER BY "Average Product Rating" DESC
  LIMIT 5
  """, engine).head()

,item_purchased,Average Product Rating
0,Gloves,3.861429
1,Sandals,3.844375
2,Boots,3.818750
3,Hat,3.801299
4,Skirt,3.784810


In [39]:
# Avg purchase amt btwn standard and express shipping

pd.read_sql_query("""
SELECT
  shipping_type,
  ROUND(AVG(purchase_amount), 2) AS "Average Purchase Amount"
  FROM shopping
  WHERE shipping_type IN ('Standard', 'Express')
  GROUP BY shipping_type
  """, engine)

,shipping_type,Average Purchase Amount
0,Express,60.48
1,Standard,58.46


In [40]:
# Avg spend btwn subscribers and non-subscribers
pd.read_sql_query("""
SELECT
  subscription_status,
  COUNT(customer_id) AS total_customers,
  ROUND(AVG(purchase_amount), 2) AS avg_spent,
  ROUND(SUM(purchase_amount), 2) AS total_revenue
  FROM shopping
  GROUP BY subscription_status
  ORDER BY total_revenue, avg_spent DESC
  """, engine)

,subscription_status,total_customers,avg_spent,total_revenue
0,Yes,1053,59.49,62645.0
1,No,2847,59.87,170436.0


In [45]:
# Products that rely on discounts to sell
pd.read_sql_query("""
SELECT item_purchased,
  ROUND(100 * SUM(CASE WHEN discount_applied = 'Yes' THEN 1 ELSE 0 END)/COUNT(*), 2) AS discount_rate
  FROM shopping
  GROUP BY item_purchased
  ORDER BY discount_rate DESC
  """, engine).head()

,item_purchased,discount_rate
0,Hat,50.0
1,Sneakers,49.0
2,Coat,49.0
3,Sweater,48.0
4,Pants,47.0


In [47]:
# Customer segmentation based on purchases
pd.read_sql_query("""
WITH customer_segment AS(
    SELECT
      customer_id,
      previous_purchases,
      CASE
        WHEN previous_purchases = 1 THEN 'NEW'
        WHEN previous_purchases BETWEEN 2 AND 4 THEN 'REGULAR'
        ELSE 'LOYAL'
      END AS customer_segment
    FROM shopping
)
SELECT customer_segment, COUNT(*) AS total_customers
FROM customer_segment
GROUP BY customer_segment
ORDER BY total_customers DESC
""", engine)

,customer_segment,total_customers
0,LOYAL,3563
1,REGULAR,254
2,NEW,83


In [49]:
# Top 3 most purchased product in each category
pd.read_sql_query("""
WITH item_counts AS (
    SELECT
        category,
        item_purchased,
        COUNT(customer_id) AS total_orders,
        ROW_NUMBER() OVER (PARTITION BY category ORDER BY COUNT(customer_id) DESC) AS item_rank
    FROM shopping
    GROUP BY category, item_purchased
    ORDER BY category, total_orders DESC
)
SELECT
    item_rank,
    category,
    item_purchased,
    total_orders
FROM item_counts
WHERE item_rank <= 3
ORDER BY category, item_rank
""", engine)

,item_rank,category,item_purchased,total_orders
0,1,Accessories,Jewelry,171
1,2,Accessories,Sunglasses,161
2,3,Accessories,Belt,161
3,1,Clothing,Pants,171
4,2,Clothing,Blouse,171
5,3,Clothing,Shirt,169
6,1,Footwear,Sandals,160
7,2,Footwear,Shoes,150
8,3,Footwear,Sneakers,145
9,1,Outerwear,Jacket,163


In [54]:
# Repeat buyers(more than 5 purchases) vs subscription
pd.read_sql_query("""
SELECT
  subscription_status,
  COUNT(customer_id) AS repeat_customers
  FROM shopping
  WHERE previous_purchases > 5
  GROUP BY subscription_status
  """, engine)

,subscription_status,repeat_customers
0,No,2518
1,Yes,958


In [53]:
# Revenue by age group
pd.read_sql_query("""
SELECT
  age_group,
  SUM(purchase_amount) AS total_revenue
  FROM shopping
  GROUP BY age_group
  ORDER BY total_revenue DESC
""", engine)

,age_group,total_revenue
0,Young Adult,62143
1,Middle-Aged,59197
2,Adult,55978
3,Senior,55763
